In [12]:
from azureml.core import Workspace

In [13]:
subscription_id = "#subscription id" # The ID of the Azure Subscription
resource_group = "fastai-practise" # Name of a logical resource group
workspace_name = "Azure_projects" # The name of the workspace to look for or to create
workspace_region = 'northcentralus' # Location of the workspace
experiment_name = 'Baby_Vibes'
score_script = 'score_and_track.py'

In [14]:
# ws = Workspace.create(name=workspace_name,
#                subscription_id=subscription_id,
#                resource_group=resource_group,
#                create_resource_group=True,
#                location=workspace_region,
#                )

In [15]:
ws = Workspace.get(name="Azure_projects", subscription_id='<your-subscription-id>', resource_group='fastai-practise')

In [16]:
from azureml.core.model import Model

model = Model.register(model_path="export.pkl",
                          model_name="Baby_Vibes",
                          tags={'data': 'Babies', 'method':'transfer learning','framework':'pytorch'},
                          description='Image classficiation of Baby Vibes',
                          workspace=ws)

Registering model Baby_Vibes


In [17]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.60','pillow==5.4.1',
                                               'torch', 'torchvision>=0.5.0'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score_and_track.py",
                                   environment=myenv)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
  - fastai==1.0.60
  - pillow==5.4.1
  - torch
  - torchvision>=0.5.0
channels:
- anaconda
- conda-forge



In [23]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'babies'

# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'Babies',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Image classficiation of Baby Vibes')

In [24]:
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Running......................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [25]:
service.get_logs()

"/usr/sbin/nginx: /azureml-envs/azureml_7e1663e048fb65ad59635b0467925068/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7e1663e048fb65ad59635b0467925068/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7e1663e048fb65ad59635b0467925068/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7e1663e048fb65ad59635b0467925068/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_7e1663e048fb65ad59635b0467925068/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-03-16T17:29:00,325356568+00:00 - gunicorn/run \n2020-03-16T17:29:00,327320662+00:00 - iot-server/run \n2020-03-16T17:29:00,329660374+00:00 - rsyslog/run \n2020-03-16T17:29:00,332583713+00:00 - nginx/run 

In [26]:
print(service.scoring_uri)

http://7724f32c-4659-4a04-823d-39ef71e678c5.westus.azurecontainer.io/score
